In [1]:
import pandas as pd

#On s'occupe d'abord de nettoyer les données sur le nombre de validation par jour 
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = 'https://www.data.gouv.fr/fr/datasets/r/63e11ecc-9d0f-40d0-bbae-5a3a403831c0'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

#On extrait les données que pour le mois de mai, à défaut d'avoir celles du mois de juillet et on fait la moyenne sur 1 jour
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'])
somme_par_station = données_validation2.groupby('LIBELLE_ARRET')['NB_VALD'].sum()
resultat_final = pd.DataFrame({'Station': somme_par_station.index, 'Nombre de validations': somme_par_station.values})
resultat_final['Nombre de validations'] = resultat_final['Nombre de validations'].div(120)


#On s'occupe ensuite de nettoyer les données sur la répartition de l'affluence sur une journée
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = 'validations-reseau-ferre-profils-horaires-par-jour-type-1er-semestre (1).csv'
données_validation= pd.read_csv(url)
données_validation = pd.read_csv(url, delimiter=';')
données_validation_station_horaire = données_validation[['LIBELLE_ARRET', 'TRNC_HORR_60','pourc_validations']]
donnees_organisees = données_validation_station_horaire.pivot_table(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations', aggfunc='mean').reset_index()

#On fusionne les données des deux tableaux pour avoir le nombre de validation sur chaque plage horaire
merged_df = pd.merge(resultat_final, donnees_organisees, left_on='Station', right_on='LIBELLE_ARRET')
columns_to_multiply = donnees_organisees.columns[1:]
resultat = merged_df.copy()
resultat[columns_to_multiply] = resultat[columns_to_multiply].multiply(resultat['Nombre de validations'], axis=0)
resultat[columns_to_multiply] = resultat[columns_to_multiply].div(100)
resultat = resultat.iloc[:, 2:]
print(resultat)

KeyboardInterrupt: 